식물 진단 context dictionary
0. 물 부족 => 나는 식물이고 물이 필요하다. I'm a plant and I need water
1. 과습 => 나는 식물이고 물을 많이 마셨다. I am a plant and I drank a lot of water.
2. 햇빛과다 => 나는 식물이고 해가 너무 뜨거워요. I'm a plant and the sun is too hot.
3. 습도 부족 => 나는 식물이고 너무 건조해요. I'm a plant and I'm too dry.
4. 높은 기온 => 나는 식물이고 너무 더워요. I'm a plant and it's too hot.
5. 낮은 기온 => 나는 식물이고 너무 추워요. I'm a plant and it's too cold.
6. 영양 부족 => 나는 식물이고 영양소 섭취가 부족해요.I'm a plant and I lack nutrients.
7. 영양 과다 => 나는 식물이고 영양소 섭취가 충분해요.I am a plant and I have enough nutrients.
8. 병해충 => 나는 식물이고 병해충이 있어요. I am a plant and I have a pest.

피드 종류 context dictionary
0. 물을 주었음 => 나는 식물이고 물을 마셨다. I am a plant and I drank water.
1. 통풍을 해줌 => 나는 식물이고 바람을 쐬었다. I am a plant and got some air.
2. 분갈이 => 나는 식물이고 오늘 다른 화분으로 이사를 했다. I am a plant and moved to another pot today.
3. 가지치기 => 나는 식물이고 오늘 머리를 잘랐다. I am a plant and I cut my hair today.
4. 분무 => 나는 식물이고 물을 마셨다. I am a plant and I drank water.
5. 영앙제 => 나는 식물이고 영양제를 먹었다. I am a plant and I took nutritional supplements.
6. 오늘의 모습 => 나는 식물이고 너가 오늘 내 사진을 찍어줬다. I am a plant and you took a picture of me today.
7. 잎 => 나는 식물이고 잎이 더 많아졌어요. I am a plant and I have more leaves.
8. 꽃 => 나는 식물이고 새 꽃이 생겼어요. I am a plant and I have a new flower.
9. 열매 => 나는 식물이고 열매가 생겼어요. I am a plant and I have fruit.

In [1]:
ko_doctor_dict = {}
ko_feed_dict = {}
eng_doctor_dict = {}
eng_feed_dict = {}

ko_doctor_dict[0] = '나는 목이 마르다.'
ko_doctor_dict[1] = '나는 물을 많이 마셨다.'
ko_doctor_dict[2] = '요즘 해가 너무 뜨거워요.'
ko_doctor_dict[3] = '공기가 너무 건조해요.'
ko_doctor_dict[4] = '날씨가 너무 더워요.'
ko_doctor_dict[5] = '날씨가 너무 추워요.'
ko_doctor_dict[6] = '나는 영양소 섭취가 부족해요.'
ko_doctor_dict[7] = '나는 영양소 섭취가 충분해요.'
ko_doctor_dict[8] = '나는 병해충이 있어요.'
ko_doctor_dict[9] = ''

ko_feed_dict[0] = '나는 물을 마셨다.'
ko_feed_dict[1] = '나는 바람을 쐬었다.'
ko_feed_dict[2] = '나는 오늘 다른 화분으로 이사를 했다.'
ko_feed_dict[3] = '나는 오늘 머리를 잘랐다.'
ko_feed_dict[4] = '나는 물을 마셨다.'
ko_feed_dict[5] = '나는 영양제를 먹었다.'
ko_feed_dict[6] = '너가 오늘 내 사진을 찍어줬다.'
ko_feed_dict[7] = '나는 잎이 더 많아졌어요.'
ko_feed_dict[8] = '나는 새 꽃이 생겼어요.'
ko_feed_dict[9] = '나는 열매가 생겼어요.'
ko_feed_dict[10] = ''

eng_doctor_dict[0] = 'but I am thirsty '
eng_doctor_dict[1] = 'but I drank a lot of water '
eng_doctor_dict[2] = 'but The sun is too hot '
eng_doctor_dict[3] = 'but I am too dry '
eng_doctor_dict[4] = 'but It is too hot '
eng_doctor_dict[5] = 'but It is too cold '
eng_doctor_dict[6] = 'but I lack nutrients '
eng_doctor_dict[7] = 'but I have enough nutrients '
eng_doctor_dict[8] = 'but I have a pest '
eng_doctor_dict[9] = ''

eng_feed_dict[0] = 'You gave me water '
eng_feed_dict[1] = 'I gave you air '
eng_feed_dict[2] = 'You moved me to a new pot '
eng_feed_dict[3] = 'You cut off my branches '
eng_feed_dict[4] = 'You sprayed me '
eng_feed_dict[5] = 'You gave me nutritional supplements '
eng_feed_dict[6] = 'You took a picture of me today '
eng_feed_dict[7] = 'I have more leaves '
eng_feed_dict[8] = 'I have a new flower '
eng_feed_dict[9] = 'I have fruit '
eng_feed_dict[10] = ''


In [2]:
# all the imports
!pip install transformers

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple


import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

<h1> 1. Dialogpt Model Load

In [ ]:
##dialogpt 모델 load

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large", padding_side = 'left')
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

trans_tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B")
trans_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B")

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.75G [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/808 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.93G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.55G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

In [ ]:
doctor_context_num = int(input(">> doctor context num :"))
feed_context_num = int(input(">> feed context num ::"))

doctor_context = eng_doctor_dict[doctor_context_num]
feed_context = eng_feed_dict[feed_context_num]



for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    context = ''
    if step == 0:
      context = f'Are you my plant?{tokenizer.eos_token} ' \
        + f'Yes, I am your plant {doctor_context}. Can you help me?{tokenizer.eos_token} '
    
    ko_input = input(">> User:")
    text_to_translate = ko_input
    model_inputs = trans_tokenizer(text_to_translate, return_tensors="pt")
    gen_tokens = trans_model.generate(**model_inputs, forced_bos_token_id=trans_tokenizer.lang_code_to_id["eng_Latn"])
    en_input = trans_tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
    
    print(f'사용자 입력 번역 : {en_input}')
    
    new_user_input_ids = tokenizer.encode(context + en_input[0] + ' How about you?'+ tokenizer.eos_token, return_tensors='pt') if step == 0 else tokenizer.encode(context + en_input[0] + tokenizer.eos_token, return_tensors='pt')

    print(f'사용 입력 encode : {new_user_input_ids[0]}')
    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    
    print(f'chat history + 사용 입력 encode : {bot_input_ids[0]}')
    
    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    bot_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    print(f'history : {tokenizer.decode(chat_history_ids[0])}')
    print(f'bot : {bot_response}')
    # context = 'I am a plant. I bloomed flowers yesterday. I drank a lot of water yesterday. '

    # print()
    # change = int(input(">> context 변경? (0 :NO  1 :YES ) :"))
    
    # if change == 1 :
    #   doctor_context_num = int(input(">> doctor context num :"))
    #   feed_context_num = int(input(">> feed context num ::"))

    #   doctor_context = eng_doctor_dict[doctor_context_num]
    #   feed_context = eng_feed_dict[feed_context_num]
      
 
    # context = f'I am a plant and {feed_context} {doctor_context}'
    
    # print(f"context : {context}")

    # temp = tokenizer.encode(context +history + tokenizer.eos_token, return_tensors='pt')

    text_to_translate = bot_response
    model_inputs = trans_tokenizer(text_to_translate, return_tensors="pt")
    gen_tokens = trans_model.generate(**model_inputs, forced_bos_token_id=trans_tokenizer.lang_code_to_id["kor_Hang"])
    ko_history = trans_tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
    # print(f"history: {history[:]}")
    print(f"Plant: {ko_history[0]}")
      
    # chat_history_ids = temp
    
    
    # print(f"Plant: {ko_history} ({context})")

In [49]:
doctor_context_num = int(input(">> doctor context num :"))
feed_context_num = int(input(">> feed context num ::"))

doctor_context = eng_doctor_dict[doctor_context_num]
feed_context = eng_feed_dict[feed_context_num]

feed_context

for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    context = ''
    if step == 0:
      context = f'Are you my plant?{tokenizer.eos_token} ' \
        + f'Yes, I am your plant {doctor_context}. Can you help me?{tokenizer.eos_token} '
    
    ko_input = input(">> User:")
    # text_to_translate = ko_input
    # model_inputs = trans_tokenizer(text_to_translate, return_tensors="pt")
    # gen_tokens = trans_model.generate(**model_inputs, forced_bos_token_id=trans_tokenizer.lang_code_to_id["eng_Latn"])
    # en_input = trans_tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
    
    # print(f'사용자 입력 번역 : {en_input}')
    
    print(f'사용자 입력 : {ko_input}')doctor_context_num = int(input(">> doctor context num :"))
feed_context_num = int(input(">> feed context num ::"))

doctor_context = eng_doctor_dict[doctor_context_num]
feed_context = eng_feed_dict[feed_context_num]



for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    context = ''
    if step == 0:
      context = f'Are you my plant?{tokenizer.eos_token} ' \
        + f'Yes, I am your plant {doctor_context}. Can you help me?{tokenizer.eos_token} '
    
    ko_input = input(">> User:")
    text_to_translate = ko_input
    model_inputs = trans_tokenizer(text_to_translate, return_tensors="pt")
    gen_tokens = trans_model.generate(**model_inputs, forced_bos_token_id=trans_tokenizer.lang_code_to_id["eng_Latn"])
    en_input = trans_tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
    
    print(f'사용자 입력 번역 : {en_input}')
    
    new_user_input_ids = tokenizer.encode(context + en_input[0] + ' How about you?'+ tokenizer.eos_token, return_tensors='pt') if step == 0 else tokenizer.encode(context + en_input[0] + tokenizer.eos_token, return_tensors='pt')

    print(f'사용 입력 encode : {new_user_input_ids[0]}')
    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    
    print(f'chat history + 사용 입력 encode : {bot_input_ids[0]}')
    
    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    bot_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    print(f'history : {tokenizer.decode(chat_history_ids[0])}')
    print(f'bot : {bot_response}')
    # context = 'I am a plant. I bloomed flowers yesterday. I drank a lot of water yesterday. '

    # print()
    # change = int(input(">> context 변경? (0 :NO  1 :YES ) :"))
    
    # if change == 1 :
    #   doctor_context_num = int(input(">> doctor context num :"))
    #   feed_context_num = int(input(">> feed context num ::"))

    #   doctor_context = eng_doctor_dict[doctor_context_num]
    #   feed_context = eng_feed_dict[feed_context_num]
      
 
    # context = f'I am a plant and {feed_context} {doctor_context}'
    
    # print(f"context : {context}")

    # temp = tokenizer.encode(context +history + tokenizer.eos_token, return_tensors='pt')

    text_to_translate = bot_response
    model_inputs = trans_tokenizer(text_to_translate, return_tensors="pt")
    gen_tokens = trans_model.generate(**model_inputs, forced_bos_token_id=trans_tokenizer.lang_code_to_id["kor_Hang"])
    ko_history = trans_tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
    # print(f"history: {history[:]}")
    print(f"Plant: {ko_history[0]}")
      
    # chat_history_ids = temp
    
    
    # print(f"Plant: {ko_history} ({context})")
    
    # new_user_input_ids = tokenizer.encode(context + en_input[0] + ' How about you?'+ tokenizer.eos_token, return_tensors='pt') if step == 0 else tokenizer.encode(context + en_input[0] + tokenizer.eos_token, return_tensors='pt')
    new_user_input_ids = tokenizer.encode(context + ko_input + ' How about you?'+ tokenizer.eos_token, return_tensors='pt') \
      if step == 0 else tokenizer.encode(context + ko_input  + tokenizer.eos_token, return_tensors='pt')

    print(f'사용 입력 encode : {new_user_input_ids[0]}')
    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    
    
    print(f'chat history + 사용 입력 encode : {bot_input_ids[0]}')
    
    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    bot_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    print(f'historfeed_contexty : {tokenizer.decode(chat_history_ids[0])}')
    # print(f'bot : {bot_response}')
    # context = 'I am a plant. I bloomed flowers yesterday. I drank a lot of water yesterday. '

    # print()
    # change = int(input(">> context 변경? (0 :NO  1 :YES ) :"))
    
    # if change == 1 :
    #   doctor_context_num = int(input(">> doctor context num :"))
    #   feed_context_num = int(input(">> feed context num ::"))

    #   doctor_context = eng_doctor_dict[doctor_context_num]
    #   feed_context = eng_feed_dict[feed_context_num]
      
 
    # context = f'I am a plant and {feed_context} {doctor_context}'
    
    # print(f"context : {context}")

    # temp = tokenizer.encode(context +history + tokenizer.eos_token, return_tensors='pt')

    text_to_translate = bot_response
    # model_inputs = trans_tokenizer(text_to_translate, return_tensors="pt")
    # gen_tokens = trans_model.generate(**model_inputs, forced_bos_token_id=trans_tokenizer.lang_code_to_id["kor_Hang"])
    # ko_history = trans_tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
    # print(f"history: {history[:]}")
    print(f"Plant: {bot_response}")
      
    # chat_histofeed_contextry_ids = temp
    
    
    # print(f"Plant: {ko_history} ({context})")

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


사용자 입력 : I gave you fresh air.
사용 입력 encode : tensor([ 8491,   345,   616,  4618,    30, 50256,  3363,    11,   314,   716,
          534,  4618,   475,   314,   716,  1165,  5894,   764,  1680,   345,
         1037,   502,    30, 50256,   314,  2921,   345,  4713,  1633,    13,
         1374,   546,   345,    30, 50256])
chat history + 사용 입력 encode : tensor([ 8491,   345,   616,  4618,    30, 50256,  3363,    11,   314,   716,
          534,  4618,   475,   314,   716,  1165,  5894,   764,  1680,   345,
         1037,   502,    30, 50256,   314,  2921,   345,  4713,  1633,    13,
         1374,   546,   345,    30, 50256])
history : Are you my plant?<|endoftext|> Yes, I am your plant but I am too dry. Can you help me?<|endoftext|> I gave you fresh air. How about you?<|endoftext|>I am a leaf on the wind.<|endoftext|>
Plant: I am a leaf on the wind.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


사용자 입력 : Is it ok?
사용 입력 encode : tensor([ 3792,   340, 12876,    30, 50256])
chat history + 사용 입력 encode : tensor([ 8491,   345,   616,  4618,    30, 50256,  3363,    11,   314,   716,
          534,  4618,   475,   314,   716,  1165,  5894,   764,  1680,   345,
         1037,   502,    30, 50256,   314,  2921,   345,  4713,  1633,    13,
         1374,   546,   345,    30, 50256,    40,   716,   257, 12835,   319,
          262,  2344,   764, 50256,  3792,   340, 12876,    30, 50256])
history : Are you my plant?<|endoftext|> Yes, I am your plant but I am too dry. Can you help me?<|endoftext|> I gave you fresh air. How about you?<|endoftext|>I am a leaf on the wind.<|endoftext|>Is it ok?<|endoftext|>I am a leaf on the wind.<|endoftext|>
Plant: I am a leaf on the wind.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


사용자 입력 : 
사용 입력 encode : tensor([50256])
chat history + 사용 입력 encode : tensor([ 8491,   345,   616,  4618,    30, 50256,  3363,    11,   314,   716,
          534,  4618,   475,   314,   716,  1165,  5894,   764,  1680,   345,
         1037,   502,    30, 50256,   314,  2921,   345,  4713,  1633,    13,
         1374,   546,   345,    30, 50256,    40,   716,   257, 12835,   319,
          262,  2344,   764, 50256,  3792,   340, 12876,    30, 50256,    40,
          716,   257, 12835,   319,   262,  2344,   764, 50256, 50256])
history : Are you my plant?<|endoftext|> Yes, I am your plant but I am too dry. Can you help me?<|endoftext|> I gave you fresh air. How about you?<|endoftext|>I am a leaf on the wind.<|endoftext|>Is it ok?<|endoftext|>I am a leaf on the wind.<|endoftext|><|endoftext|>I am a leaf on the wind.<|endoftext|>
Plant: I am a leaf on the wind.


In [ ]:
chat_history_ids[0]

tensor([   40,   716,   257,  4618,   290,   314, 24070,  1660,    13, 10814,
          612, 50256])

2<h1> 2. Pleon Model Load

In [ ]:
##pleon 모델 load

from transformerfeed_contexts import AutoModelWithLMHead, AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('/content/drive/Othercomputers/MacBookPro/chatbot/ko_dialogpt/checkpoint-300000', padding_side='left')
model = AutoModelWithLMHead.from_pretrained('/content/drive/Othercomputers/MacBookPro/chatbot/ko_dialogpt/checkpoint-300000')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1116: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    tokenizer.encode
    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
feed_context
    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1000,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,

        temperature=0.8
    )

    context = f('I am a plant and' {feed_context})
    
    chat_history_ids = tokenizer.encode(context +history + tokenizer.eos_token, return_tensors='pt')

    print("Plant: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:안녕~


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: 나는 요즘 회사 일이 바빠서 운동을 못해
>> User:많이 바빠?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: 나는 지금 바빠 키키
>> User:요즘 회사일 일이 많이 바쁘다며


...

Plant: 싶네요 키키 아휴 키키 뭐라도 챙겨먹어
>> User:나 지금 닭발 먹을거야


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: ##물떡은 너무 질겨
>> User:물떡 맛있어.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: ##바이러스가 유행이래
